<a href="https://colab.research.google.com/github/kchenTTP/python-series/blob/main/object_oriented_programming_in_python/Object_Oriented_Programming_in_Python_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Object Oriented Programming in Python - Part 2**

In the previous class, we covered the basics of object-oriented programming. In this class, we are going to dive deeper into more OOP features in Python that will help you write more efficient and maintainable code.

**Table of Contents**

- [Class Methods & Static Methods](#scrollTo=ctaC5CoFZyBn)
  - [Class Methods](#scrollTo=S1U_oQzvmpRR)
  - [Static Methods](#scrollTo=9Uyx1Oj9vOq_)
- [Special Methods & Attributes](#scrollTo=K3MSR-8x31WE)
  - [Dunder Methods](#scrollTo=GMP_yJmZDoWH)
  - [Special Attributes](#scrollTo=FqSGZ48CVDXN)
- [OOP Relationships](#scrollTo=4AzmBqOAF8lm)
  - [Inheritance](#scrollTo=qPlMEcmqYAII)
  - [Composition](#scrollTo=ueSxzCZ_ZbaS)
  - [Polymorphism](#scrollTo=UnQes5T8ZfXl)
- [Advanced Concepts (Additional Material)](#scrollTo=B4diOn6q9fQP)


## **Class Methods & Static Methods**

In the previous class, we explored instance methods, which work with data stored in an instance of a class. These methods take `self` as the first argument, referring to the specific instance. In this class, we'll dive into two other types of methods: class methods and static methods.


### **Class Methods**

Class methods are methods that work with the class itself rather than instances of the class. They are decorated with `@classmethod` and receive the class as their first parameter (conventionally called `cls`).


Here are a few reasons for using class methods.

<br>

**Managing Class-Level State**

Class methods can modify or retrieve class-level state that applies to all instances.


In [1]:
class Employee:
  company: str = "Tech Corp"
  active_employees: int = 0

  def __init__(self, name, salary):
    self.name: str = name
    self.salary: float = salary
    Employee.active_employees += 1

  def __del__(self):
    Employee.active_employees -= 1

  @classmethod
  def set_company(cls, new_company):
    cls.company = new_company

  @classmethod
  def get_active_employees(cls):
    return cls.active_employees

  @classmethod
  def get_company(cls):
    return cls.company

# Using class methods
print(f"Company: {Employee.get_company()}")
Employee.set_company("New Corp")
print(f"New Company: {Employee.get_company()}")
print("---------")

Tom = Employee("Tom", 50000)
Alice = Employee("Jenna", 60000)
print(f"Active Employees: {Employee.get_active_employees()}")

del Alice
print(f"Active Employees: {Employee.get_active_employees()}")

Company: Tech Corp
New Company: New Corp
---------
Active Employees: 2
Active Employees: 1


**Factory Methods for Object Creation**

Sometimes you need different ways to create objects from a class. Class methods can be used as an alternative constructor.

> 📒 **Note:** Here, our class method returns a `cls` (the class) instance, providing an alternative way to create an object.


In [2]:
class Date:
  def __init__(self, year: int, month: int, day: int):
    self.year = year
    self.month = month
    self.day = day

  @property
  def date(self):
    return f"{self.year}-{self.month}-{self.day}"

  @classmethod
  def from_string(cls, date_string: str) -> "Date":
    try:
      year, month, day = map(int, date_string.split('-'))
    except ValueError:
      raise ValueError("Invalid date format. Use 'YYYY-MM-DD'.")
    return cls(year, month, day)

# Example
date1 = Date(2024, 1, 15)   # Using __init__
date2 = Date.from_string("2024-1-15")  # Using alternate constructor

print(date1.date)
print(date2.date)

2024-1-15
2024-1-15


#### **Examples (Additional Material)**

Now let's look at some real world examples at how you could use class methods


**Role-Based Access Control in a User Management System**


In [152]:
from typing import Literal

class User:
  roles: dict[str, list] = {"admin": [], "editor": [], "viewer": []}
  permissions: dict[str, list[Literal["create", "read", "update", "delete"]]] = {
      "admin": ["create", "read", "update", "delete"],
      "editor": ["read", "update"],
      "viewer": ["read"],
    }

  def __init__(self, username: str, role: str = "viewer"):
    if role not in User.roles:
      raise ValueError(f"Invalid role: {role}")
    self.username = username
    self.role = role
    User.roles[role].append(username)

  @classmethod
  def show_roles(cls):
    print(cls.roles)

  @classmethod
  def get_users_by_role(cls, role: str) -> list:
    if role not in cls.roles:
      raise ValueError(f"Role '{role}' does not exist.")
    return cls.roles[role]

  def has_permission(self, action: Literal["create", "read", "update", "delete"]) -> bool:
    return action in User.permissions.get(self.role, [])

  def perform_action(self, action: Literal["create", "read", "update", "delete"]):
    if self.has_permission(action):
      return f"{self.username} can {action}."
    else:
      return f"{self.username} cannot {action} due to insufficient permissions."

admin = User("Timothy", "admin")
editor = User("Jane", "editor")
viewer = User("Nora", "viewer")
User.show_roles()
print(User.get_users_by_role("admin"))
print("----------")

# Perform actions based on permissions
print(admin.perform_action("create"))
print(editor.perform_action("create"))
print(viewer.perform_action("read"))
print(viewer.perform_action("update"))

{'admin': ['Timothy'], 'editor': ['Jane'], 'viewer': ['Nora']}
['Timothy']
----------
Timothy can create.
Jane cannot create due to insufficient permissions.
Nora can read.
Nora cannot update due to insufficient permissions.


**Caching Mechanism**

> 💡 *Caching is a technique used to store a copy of data in a temporary storage location (called a **cache**) so that it can be accessed more quickly in the future.*


In [4]:
class URLShortener:
  cache = {}

  def __init__(self, long_url):
    self.long_url = long_url
    self.short_url = self._generate_short_url()
    URLShortener.cache[self.short_url] = long_url

  def _generate_short_url(self):
    """Generates a simple short URL."""
    return f"short.ly/{hash(self.long_url) % 10000}"

  @classmethod
  def get_long_url(cls, short_url):
    """Fetch the original URL from the cache."""
    return cls.cache.get(short_url, "Not found")

  @classmethod
  def cache_size(cls):
    """Returns the size of the cache."""
    return len(cls.cache)

# Example Usage
url1 = URLShortener("https://example.com/long-url")
url2 = URLShortener("https://anotherexample.com/very-long-url")

print(url1.short_url)
print(URLShortener.get_long_url(url1.short_url))
print(URLShortener.get_long_url("skdjksdjf"))
print(URLShortener.cache_size())

short.ly/4885
https://example.com/long-url
Not found
2


### **Static Methods**

Static methods are methods that don't operate on either the instance or the class. They are decorated with `@staticmethod` and don't receive any special first parameter.

> 📒 **Note:** Static methods are like utility functions that don't require access to the class or instance.


In [15]:
class MathUtils:
  @staticmethod
  def add(x, y):
    return x + y

  @staticmethod
  def subtract(x, y):
    return x - y

  @staticmethod
  def multiply(x, y):
    return x * y

  @staticmethod
  def divide(x, y):
    if y == 0:
      raise ValueError("Division by zero is not allowed.")
    return x / y

  @staticmethod
  def power(x, y):
    return x ** y

# Using static methods
print(MathUtils.add(5, 3))
print(MathUtils.power(10, 2))

8
100


In [14]:
class Date:
  @staticmethod
  def is_valid_date(year: int, month: int, day: int) -> bool:
    # Type validation
    if not isinstance(year, int) or not isinstance(month, int) or not isinstance(day, int):
      raise TypeError("All arguments must be integers")

    # Year validation
    if year < 1:
      return False

    # Month validation
    if month < 1 or month > 12:
      return False

    # Day validation
    if day < 1 or day > 31:
      return False
    if month in [4, 6, 9, 11] and day > 30:
      return False
    if month == 2:
      if Date.is_leap_year(year):
        return day <= 29
      else:
        return day <= 28

    return True

  @staticmethod
  def is_leap_year(year: int) -> bool:
    # Type validation
    if not isinstance(year, int):
      raise TypeError("Year must be an integer")

    if year % 4 == 0:
      if year % 100 == 0:
        return year % 400 == 0
      return True
    return False

# Using static methods
print(Date.is_valid_date(2024, 1, 15))
print(Date.is_leap_year(2024))
print(Date.is_valid_date(-100, 1, 1))

True
True
False


In [16]:
class Temperature:
  def __init__(self, *, celcius: float | None = None, fahrenheit: float | None = None, kelvin: float | None = None):
    # Check if one and only one of the temperature units is provided
    if not celcius and not fahrenheit and not kelvin:
      raise ValueError("At least one temperature unit must be provided")
    elif bool(celcius) + bool(fahrenheit) + bool(kelvin) > 1:
      raise ValueError("Only one temperature unit can be provided")

    if celcius:
      self.celcius = celcius
      self.fahrenheit = self.celcius_to_fahrenheit(celcius)
      self.kelvin = self.celcius_to_kelvin(celcius)
    if fahrenheit:
      self.fahrenheit = fahrenheit
      self.celcius = self.fahrenheit_to_celcius(fahrenheit)
      self.kelvin = self.fahrenheit_to_kelvin(fahrenheit)
    if kelvin:
      self.kelvin = kelvin
      self.celcius = self.kelvin_to_celcius(kelvin)
      self.fahrenheit = self.kelvin_to_fahrenheit(kelvin)

  @staticmethod
  def celcius_to_fahrenheit(celcius: float) -> float:
    return (celcius * 9/5) + 32

  @staticmethod
  def fahrenheit_to_celcius(fahrenheit) -> float:
    return (fahrenheit - 32) * 5/9

  @staticmethod
  def celcius_to_kelvin(celcius: float) -> float:
    return celcius + 273.15

  @staticmethod
  def fahrenheit_to_kelvin(fahrenheit: float) -> float:
    return Temperature.celcius_to_kelvin(Temperature.fahrenheit_to_celcius(fahrenheit))

  @staticmethod
  def kelvin_to_celcius(kelvin: float) -> float:
    return kelvin - 273.15

  @staticmethod
  def kelvin_to_fahrenheit(kelvin: float) -> float:
    return Temperature.celcius_to_fahrenheit(Temperature.kelvin_to_celcius(kelvin))

# Using static methods
celcius = 25.0

print(f"Fahrenheit: {Temperature.celcius_to_fahrenheit(celcius)}")
print(f"Kelvin: {Temperature.celcius_to_kelvin(celcius)}")

Fahrenheit: 77.0
Kelvin: 298.15


## **Special Methods & Attributes**

Python provides several special methods (also called **magic** methods or **dunder** methods) and attributes  that enable advanced functionality and customization for your classes. These methods start and end with ***double underscores*** (`__`) and are used internally by Python, but you can override them to define your class's behavior.


### **Dunder Methods**


#### **Object Initialization and Destruction**

**`__init__`**

  - Used as the ***constructor*** of the class to initialize attributes.
  - Called automatically when an object is created.

**`__del__`**

  - Used as the ***destructor*** of the class.
  - Called when an object is about to be destroyed (rarely used).


In [2]:
class Creature:
  def __init__(self, name: str):
    self.name = name
    print(f"{self.name} has been created.")

  def __del__(self):
    print(f"{self.name} has been destroyed.")

# Example
goblin = Creature("Goblin")
del goblin

Goblin has been created.
Goblin has been destroyed.


#### **Object Representation**

**`__str__`**

  - Defines the human-readable string representation of an object.
  - Used by `str()` and `print()`.

**`__repr__`**

  - Defines the official string representation of an object.
  - Used by `repr()` and in debugging.
  - If `__str__` is not defined, `str()` and `print()` will use `__repr__` instead.


In [23]:
class Person:
  def __init__(self, name, age):
    self.name = name
    self.age = age

  def __str__(self):
    return f"{self.name}, {self.age} years old"

  def __repr__(self):
    return f"Person(name={self.name!r}, age={self.age!r})"

# Classes without string representation
print(Creature)
print(str(Creature))
print("--------")

# Classes with string representation
person = Person("Jeffery", 25)
print(person)
print(str(person))
print(repr(person))

<class '__main__.Creature'>
<class '__main__.Creature'>
--------
Jeffery, 25 years old
Jeffery, 25 years old
Person(name='Jeffery', age=25)


#### **Operator Overloading**

##### **Arithmetic Operators**

**`__add__`**, **`__sub__`**, **`__mul__`**, **`__truediv__`**, **`__floordiv__`**, **`__mod__`**, **`__pow__`**

  - Define the behavior of arithmetic operators: `+`, `-`, `*`, `/`, `//`, `%`, `**`.

##### **Comparison Operators**

**`__eq__`**, **`__ne__`**, **`__lt__`**, **`__gt__`**, **`__le__`**, **`__ge__`**

  - Define the behavior of comparison operators: `==`, `!=`, `<`, `>`, `<=`, `>=`.

> ❗ When overloading operators, the dunder methods should only recieve `self` and `other` as the first two arguments.


In [130]:
from typing import Union

class MyString:
  def __init__(self, value: str):
    self.value = value

  # Arithmetic Operators
  def __add__(self, other: Union["MyString", str]) -> "MyString":
    if isinstance(other, MyString):
      return MyString(self.value + other.value)
    elif isinstance(other, str):
      return MyString(self.value + other)
    else:
      raise TypeError

  def __mul__(self, other: int) -> "MyString":
    if isinstance(other, int):
      return MyString(self.value * other)
    else:
      raise TypeError

  # Comparison Operators
  def __eq__(self, other: Union["MyString", str]) -> bool:
    if isinstance(other, MyString):
      return self.value == other.value
    elif isinstance(other, str):
      return self.value == other

  def __ne__(self, other: Union["MyString", str]) -> bool:
    if isinstance(other, MyString):
      return self.value != other.value
    elif isinstance(other, str):
      return self.value != other

  # Representation Methods
  def __str__(self):
    return self.value

  def __repr__(self):
    return f"MyString('{self.value}')"

# Example
s1 = MyString("Hello")
s2 = MyString("World")

print(s1 + " " + s2)
print(s1 * 3)
print(s1 == "Good morning")
print(s1 != s2)

Hello World
HelloHelloHello
False
True


#### **Container Behavior**

**`__len__`**

  - Defines the behavior of `len()`.

**`__getitem__`**

  - Defines behavior for indexing (e.g., `obj[key]`).

**`__setitem__`**

  - Defines behavior for assignment to indexed elements.

**`__delitem__`**

  - Defines behavior for deleting indexed elements.

**`__iter__`**

  - Makes an object iterable (used in loops).

**`__contains__`**

  - Defines the behavior of the `in` operator.


In [63]:
from typing import Any

class MyList:
  def __init__(self, *items: Any):
    self._length = 0

    # Dynamically setting attributes base on the number of arguements passed in when initialized
    for i, item in enumerate(items):
      setattr(self, f"{i}", item)
      self._length += 1

  def __len__(self):
    return self._length

  def __iter__(self):
    for i in range(self._length):
      yield getattr(self, f"{i}")

  def __contains__(self, item: Any):
    for i in range(self._length):
      if getattr(self, f"{i}") == item:
        return True
    return False

  def __str__(self):
    return f"{[getattr(self, f'{i}') for i in range(self._length)]}"

  def __getitem__(self, index: int):
    if not isinstance(index, int):
      raise TypeError("Index must be an integer")
    if index < 0 - self._length or index >= self._length:
      raise IndexError("Index out of range")

    # Convert negative index to positive
    if index < 0:
      index = self._length + index

    # Dynamically getting attributes
    for i in range(self._length):
      if i == index:
        return getattr(self, f"{i}")

  def __setitem__(self, index: int, value: Any):
    if not isinstance(index, int):
      raise TypeError("Index must be an integer")
    if index < 0 - self._length or index >= self._length:
      raise IndexError("Index out of range")

    # Convert negative index to positive
    if index < 0:
      index = self._length + index

    # Dynamically setting attributes
    for i in range(self._length):
      if i == index:
        setattr(self, f"{i}", value)
        break

  def __delitem__(self, index: int):
    if not isinstance(index, int):
      raise TypeError("Index must be an integer")
    if index < 0 - self._length or index >= self._length:
      raise IndexError("Index out of range")

    # Convert negative index to positive
    if index < 0:
      index = self._length + index

    # Dynamically deleting attributes and adjust the rest of the attributes and length of object
    for i in range(self._length):
      if i == index:
        delattr(self, f"{i}")
      if i > index:
        setattr(self, f"{i - 1}", getattr(self, f"{i}"))
        delattr(self, f"{i}")
    self._length -= 1

# Example
lst = MyList(1, 2, 3, 4, 5)

print(lst)
print(2 in lst)
print(20 in lst)
print("---------")

print(f"Index 0: {lst[0]}")
lst[0] = 10
print(f"New Index 0: {lst[0]}")
print("---------")

print(f"Length: {len(lst)}")
del lst[-2]
print(f"Length After Removing 1 Item: {len(lst)}")
print("---------")

for item in lst:
  print(item)

[1, 2, 3, 4, 5]
True
False
---------
Index 0: 1
New Index 0: 10
---------
Length: 5
Length After Removing 1 Item: 4
---------
10
2
3
5


> 📒 **Note:** `setattr`, `getattr`, and `delattr` are built-in functions that provide dynamic access to an object's attributes.


#### **Context Manager**

**`__enter__`**

  - Executes when entering the `with` block.
  - Performs any necessary setup or initialization.
  - Returns the object to be used within the `with` block.
    - Often returns `self` to provide access to the instance.

**`__exit__`**

  - Executes when exiting the `with` block (whether normally or due to an exception).
  - Handles cleanup and resource release.
  - Receives exception information if one occurred:
    - `exc_type`: Type of exception (e.g., `ValueError`, `TypeError`). `None` if no exception occurred.
    - `exc_val`: The exception instance with error details. `None` if no exception occurred.
    - `exc_tb`: Traceback object containing the call stack. `None` if no exception occurred.
  - Return value controls exception propagation:
    - `True`: Suppresses the exception.
    - `False` or None: Allows the exception to propagate.


In [72]:
import time

class Timer:
  def __enter__(self):
    self.start = time.perf_counter()
    return self

  def __exit__(self, exc_type, exc_val, exc_tb):
    self.end = time.perf_counter()
    self.duration = self.end - self.start
    print(f"Execution time: {self.duration:.4f} seconds")

# Example
with Timer():
    sum(range(1000000))

Execution time: 0.0427 seconds


### **Special Attributes**

**`__dict__`**

A special attribute that stores the object's writable attributes as key-value pairs in a dictionary.

**`__class__`**

A special attribute that holds a reference to the class of the object.

**`__doc__`**

A special attribute that contains the docstring of the class, method, or function.


In [149]:
class MyClass:
  """😀 This is a simple class with a docstring."""

  cls_attr = 100

  def __init__(self, value):
    self.value = value

  def my_method(self):
    """🐍 This is a method with a docstring."""
    pass

obj = MyClass(42)

print(obj.__dict__)
print(MyClass.__dict__)
print(obj.__class__)
print(obj.__doc__)
print(obj.my_method.__doc__)

{'value': 42}
{'__module__': '__main__', '__doc__': '😀 This is a simple class with a docstring.', 'cls_attr': 100, '__init__': <function MyClass.__init__ at 0x7a19aeca3f60>, 'my_method': <function MyClass.my_method at 0x7a19aeca2ca0>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>}
<class '__main__.MyClass'>
😀 This is a simple class with a docstring.
🐍 This is a method with a docstring.


There are many more dunder methods and special attributes beyond those covered here, each serving specific purposes to customize and extend the behavior of objects in Python. Understanding and using these methods effectively can help you write cleaner, more Pythonic code.


### **More Examples (Additional Material)**


**Circle**


In [22]:
class Circle:
  def __init__(self, radius: float):
    self.radius = radius

  @property
  def area(self):
    return 3.14 * (self.radius ** 2)

  # Comparison Operators
  def __eq__(self, other: "Circle"):
    if not isinstance(other, Circle):
      return False
    return self.area == other.area

  def __ne__(self, other: "Circle"):
    if not isinstance(other, Circle):
      return False
    return self.area != other.area

  def __lt__(self, other: "Circle"):
    if not isinstance(other, Circle):
      return False
    return self.area < other.area

  def __le__(self, other: "Circle"):
    if not isinstance(other, Circle):
      return False
    return self.area <= other.area

  def __gt__(self, other: "Circle"):
    if not isinstance(other, Circle):
      return False
    return self.area > other.area

  def __ge__(self, other: "Circle"):
    if not isinstance(other, Circle):
      return False
    return self.area >= other.area

# Example
small_circle = Circle(10)
big_circle = Circle(20)

print(small_circle == big_circle)
print(small_circle != big_circle)
print(small_circle < big_circle)

False
True
True


**Game Level**


In [126]:
import random

class LevelConfig:
  def __init__(self, level_name: str):
    self.level_name = level_name
    self.objects = ["💰 treasure chest", "🍾 health potion", "🪖 armor", "☠️ poison trap"]
    self.monsters = ["👺 goblin", "🧌 orc", "🐉 dragon"]

class LevelLayout:
  def __init__(self, config: LevelConfig):
    self.config = config
    self.layout = []
    self.size = 5  # 5x5 grid
    self.cell_width = 0
    for item in self.config.objects + self.config.monsters:
      self.cell_width = max(self.cell_width, len(item))
    self.initialize_empty_layout()

  def initialize_empty_layout(self):
    self.layout = [["empty" for _ in range(self.size)] for _ in range(self.size)]

  def randomize(self):
    self.initialize_empty_layout()
    # Place random objects
    for obj in self.config.objects:
      x, y = random.randint(0, self.size-1), random.randint(0, self.size-1)
      while self.layout[y][x] != "empty":
        x, y = random.randint(0, self.size-1), random.randint(0, self.size-1)
      self.layout[y][x] = obj

    # Place random monsters
    for monster in self.config.monsters:
      x, y = random.randint(0, self.size-1), random.randint(0, self.size-1)
      while self.layout[y][x] != "empty":
        x, y = random.randint(0, self.size-1), random.randint(0, self.size-1)
      self.layout[y][x] = monster

class GameLevel:
  def __init__(self, level_name: str):
    self.level_name = level_name
    self.config = None
    self.level_layout = None

  def __enter__(self):
    print(f"Initializing level: {self.level_name}")
    self.config = LevelConfig(self.level_name)
    self.level_layout = LevelLayout(self.config)
    self.level_layout.randomize()
    return self

  def __exit__(self, exc_type, exc_val, exc_tb):
    print(f"Cleaning up level: {self.level_name}")
    self.config = None
    self.level_layout = None
    if exc_type is not None:
      print(f"An error occurred: {exc_val}")
      return True

  def display_layout(self):
    if self.level_layout is None:
      raise ValueError("Level layout has not been initialized")

    total_width = (self.level_layout.cell_width * self.level_layout.size) + (self.level_layout.size - 1)
    print(("+" + "-" * self.level_layout.cell_width) * self.level_layout.size + "+")

    for row in self.level_layout.layout:
      formatted_row = "|" + "|".join(f"{cell:^{self.level_layout.cell_width}}" for cell in row) + "|"
      print(formatted_row)
      print(("+" + "-" * self.level_layout.cell_width) * self.level_layout.size + "+")

In [127]:
with GameLevel("Dungeon") as game:
  print(f"Level Name: {game.level_name}")
  print(f"Level Config: {game.config.__dict__}")
  print()
  game.display_layout()

Initializing level: Dungeon
Level Name: Dungeon
Level Config: {'level_name': 'Dungeon', 'objects': ['💰 treasure chest', '🍾 health potion', '🪖 armor', '☠️ poison trap'], 'monsters': ['👺 goblin', '🧌 orc', '🐉 dragon']}

+----------------+----------------+----------------+----------------+----------------+
|    🪖 armor     |     empty      |     empty      |    🐉 dragon    |     empty      |
+----------------+----------------+----------------+----------------+----------------+
|     empty      |     empty      |     empty      |     empty      |     empty      |
+----------------+----------------+----------------+----------------+----------------+
|     empty      |     🧌 orc      |     empty      |     empty      |    👺 goblin    |
+----------------+----------------+----------------+----------------+----------------+
|🍾 health potion |     empty      |     empty      |💰 treasure chest|     empty      |
+----------------+----------------+----------------+----------------+----------------+


## **Object-Oriented Programming Relationships**

...

### **Inheritance**

Inheritance allows a class to inherit attributes and methods from another class.

...


### **Composition**

Composition is a way to combine simple objects or data types into more complex ones.

...


### **Polymorphism**

Polymorphism allows us to use a single interface with different underlying forms (data types or classes).

...


## **Final Example: ...**

...

**Overview**

1. ...
2. ...
3. ...


## **Advanced Concepts (Additional Material)**

...

### **Abstract Base Class (ABC)**

...


### **Metaclass**

...


## **Conclusion**

This concludes our lessons on object-oriented programming in python ...



- Understanding the problems OOP solves
- Creating and working with classes and objects
- Managing data through attributes
- Defining behavior with methods
- Controlling access with properties
